In [4]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = get ( url )
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [6]:
df = df[df.Borough != 'Not assigned']

In [7]:
df = df.groupby('Postal Code').sum()

In [8]:
df

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


In [9]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e0439a5df9504d65b16e8a3346e95b3c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='YbYJMcqT8QFoL_bR7yD_aaYd03C1XMAmCw-CQ6Oo2Ocg',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_e0439a5df9504d65b16e8a3346e95b3c.get_object(Bucket='courseracaptone1-donotdelete-pr-ikzaa3mhxewund',Key='Geospatial_Coordinates (3).csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

pd_geo = pd.read_csv(body)
pd_geo


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [10]:
pd_post_geo = pd.merge(pd_geo, df, on='Postal Code')

pd_post_geo = pd_post_geo[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]

In [11]:
pd_post_geo

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [29]:
!pip install folium

     |████████████████████████████████| 94 kB 7.4 MB/s  eta 0:00:01


In [30]:
import folium
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [16]:
pd_post_geo.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
geo_group = pd_post_geo.groupby('Neighbourhood').mean().reset_index()
latitude = geo_group['Latitude'].values
longitude = geo_group['Longitude'].values
geo_group

,Neighbourhood,Latitude,Longitude
0,Agincourt,43.794200,-79.262029
1,"Alderwood, Long Branch",43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,Bayview Village,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
...,...,...,...
94,"Willowdale, Willowdale West",43.782736,-79.442259
95,Woburn,43.770992,-79.216917
96,Woodbine Heights,43.695344,-79.318389
97,York Mills West,43.752758,-79.400049


In [21]:
KClusters = 5
geo_train = geo_group.drop('Neighbourhood', axis=1)
kmeans = KMeans(n_clusters = KClusters, random_state = 41 ).fit(geo_train)
geo_group['labels'] = kmeans.labels_

In [22]:
geo_group

,Neighbourhood,Latitude,Longitude,labels
0,Agincourt,43.794200,-79.262029,3
1,"Alderwood, Long Branch",43.602414,-79.543484,1
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,4
3,Bayview Village,43.786947,-79.385975,4
4,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,4
...,...,...,...,...
94,"Willowdale, Willowdale West",43.782736,-79.442259,4
95,Woburn,43.770992,-79.216917,3
96,Woodbine Heights,43.695344,-79.318389,0
97,York Mills West,43.752758,-79.400049,4


In [31]:
map_clusters = folium.Map(location=[40.7127281, -74.0060152], zoom_start=11)
# set color scheme for the clusters
x = np.arange(KClusters)
ys = [i + x + (i*x)**2 for i in range(KClusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(geo_group['Latitude'], geo_group['Longitude'], geo_group['Neighbourhood'], geo_group['labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters